In [ ]:
!pip install spotipy
import spotipy
import os
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 5.5 MB/s eta 0:00:00


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Connect to Spotify API using SpotifyClientCredentials

In [ ]:
# Load client id and client secret
CLIENT_ID="2b2ef724f49f4819984d1093c9df7f75"
CLIENT_SECRET="e50dbae21a54489fb63ed1a21106b66a"

# Connect with Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# Load artist's ids
artist_id_david_crosby = '59zdhVoWxSoHMc74n098Re'
artist_id_stephen_stills = '4WlSvDKaq1PA2Nr7cCIPxX'
artist_id_nash_graham = '2E6Roj0oQnJIm2BeXwDica'
artist_id_neil_young = '6v8FB84lnmJs434UJf2Mrm'
artist_id_cnsy = '1CYsQCypByMVgnv17qsSbQ'
artist_id_csn = '2pdvghEHZJtgSXZ7cvNLou'
artist_id_cn = '2mDlls63YpRsMjgpyXxyu6'

### Get all albums from the given artist

In [ ]:
# Get artist's albums
all_albums = []
albums_and_singles = []

def get_all_albums(artist_id, artist_name):
    albums = sp.artist_albums(artist_id, limit=50) # Fetch albums from Spotify for the given artist with the specified album type, with a limit of 50 results per request

    all_albums.extend(albums['items']) # Add fetched album to the all_albums

    while albums['next']:
        albums = sp.next(albums) # Get the next set of albums
        all_albums.extend(albums['items']) # Add fetched album to the all_albums

    # ***Note***: Sometimes, Spotify may return inconsistent results, so further filtering methods are recommended to avoid unwanted data, such as albums from collaborations or other artist with the same name
    filtered_albums = [album for album in all_albums if album['album_type'] == 'album' and album['artists'][0]['name'].lower() == artist_name.lower()] # Filter albums to ensure only albums with album_type = 'album' and artists with provided name are stored
    filtered_singles = [album for album in all_albums if album['album_type'] == 'single' and album['artists'][0]['name'].lower() == artist_name.lower()] # Filter albums to ensure only albums with album_type = 'single' and artist with provided name are stored
    filtered_compilations = [album for album in all_albums if album['album_type'] == 'compilation' and album['artists'][0]['name'].lower() == artist_name.lower()] # Filter albums to ensure only albums with album_type = 'single' and artist with provided name are stored

    for album in filtered_albums:
        print(album['name'], album['album_type'])
    print("##################################################")
    for album in filtered_singles:
        print(album['name'], album ['album_type'])
    print("##################################################")
    for album in filtered_compilations:
        print(album['name'], album ['album_type'])

    albums_and_singles = filtered_albums # + filtered_singles # Concatenate lists of albums and singles to get a complete result
    return albums_and_singles

albums_and_singles = get_all_albums(artist_id_stephen_stills, 'Stephen Stills') # Example usage of the function

### Get basic information about artist's albums

In [ ]:
# Get album id, type, name, release date, performer for given artist id
all_albums_info = []

# Iterate through list of albums_and_singles and fetch information about album
for album in albums_and_singles:
    info = {
        'album_type' : album.get('album_type'),
        'artists_ids' : [artist.get('id') for artist in album.get('artists', [])],
        'artists_names' : [artist.get('name') for artist in album.get('artists', [])],
        'album_id' : album.get('id'),
        'album_name' : album.get('name'),
        'album_release_date' : album.get('release_date')
    }

    all_albums_info.append(info) # Append collected information to all_albums_info

### Get track popularity score

In [ ]:
# Function to extract track popularity from given song
def get_track_popularity(track_id):
    track_info = sp.track(track_id)
    track_popularity = track_info['popularity']

    return track_popularity

### Get all tracks from an album and corresponding basic track information

In [ ]:
def get_all_tracks_from_album(album_id, album_info):
    all_tracks_info = []

    tracks = sp.album_tracks(album_id)  # Get set of tracks
    all_tracks = tracks['items'] # Extract the 'items', which contain the track information

    while tracks['next']:
        tracks = sp.next(tracks) # Fetch the next batch of tracks
        all_tracks.extend(tracks['items']) # Append the new batch of tracks

    # Iterate through all_tracks, get track popularity index and other basic information and store it into dictionary 'info'
    for track in all_tracks:
      track_popularity = get_track_popularity(track['id'])
      for artist in track['artists']:
            info = {
                'artist_id': artist['id'],
                'artist_name': artist['name'],
                'duration_ms': track['duration_ms'],
                'track_id': track['id'],
                'track_name': track['name'],
                'popularity': track_popularity,
                'album_id': album_info['album_id'],
                'album_name': album_info['album_name'],
                'album_release_date': album_info['album_release_date'],
                'album_type': album_info['album_type']
            }
            all_tracks_info.append(info) # Append collected information to all_tracks_info

    return all_tracks_info # Return all_tracks_info

for album in all_albums_info:
    get_all_tracks_from_album(album['album_id'], album) # Call function for each album in all_albums_info

### Get all audio features from a track

In [ ]:
# Function to extract audio features from tracks
def get_audio_features_from_track(all_tracks_info):
    audio_features_list = []
    track_ids = [track_info['track_id'] for track_info in all_tracks_info] # Extract track_id for each track

    # Iterate through the track_ids in batches of 100 to avoid hitting Spotify API's rate limits
    for i in range(0, len(track_ids), 100):
            batch = track_ids[i:i+100] # Create a batch of 100 track ids
            audio_features = sp.audio_features(batch) # Fetch audio features from the current track
            for features_dict in audio_features: # Iterate through each dictionary of audio_features
                if features_dict:
                    features = {
                        'danceability' : features_dict['danceability'],
                        'energy': features_dict['energy'],
                        'key': features_dict['key'],
                        'loudness': features_dict['loudness'],
                        'mode': features_dict['mode'],
                        'speechiness': features_dict['speechiness'],
                        'acousticness': features_dict['acousticness'],
                        'instrumentalness': features_dict['instrumentalness'],
                        'liveness': features_dict['liveness'],
                        'valence': features_dict['valence'],
                        'tempo': features_dict['tempo'],
                        'track_id' : features_dict['id'],
                        'duration_ms' : features_dict['duration_ms'],
                        'time_signature' : features_dict['time_signature']
                    }
                audio_features_list.append(features)
            time.sleep(1) # Pause for 1 second between each API request to avoid hitting rate limits
    return audio_features_list

#audio_features = get_audio_features_from_track(all_tracks_info) # Call the function to extract audio features for all tracks

## Stephen Stills - additional albums

In [ ]:
# Get basic album information for Stephen Stills additional albums
man_alive = ['6nIuJOOeQf1OGpp7zzZWVx']
man_alive_info = []
super_session = ['6bdy2PnssuzDkldvAOjVmj']
super_session_info = []

def fetch_album_info(album_ids):
    album_info_list = []
    for album_id in album_ids:
        album = sp.album(album_id)
        info = {
            'album_type': album.get('album_type'),
            'artists_ids': [artist.get('id') for artist in album.get('artists', [])],
            'artists_names': [artist.get('name') for artist in album.get('artists', [])],
            'album_id': album.get('id'),
            'album_name': album.get('name'),
            'album_release_date': album.get('release_date')
        }
        album_info_list.append(info)
    return album_info_list

man_alive_info = fetch_album_info(man_alive)
super_session_info = fetch_album_info(super_session)

print("Man Alive Info:", man_alive_info)
print("Super Session Info:", super_session_info)

Man Alive Info: [{'album_type': 'album', 'artists_ids': ['4WlSvDKaq1PA2Nr7cCIPxX'], 'artists_names': ['Stephen Stills'], 'album_id': '6nIuJOOeQf1OGpp7zzZWVx', 'album_name': 'Man Alive!', 'album_release_date': '2005-08-09'}]
Super Session Info: [{'album_type': 'album', 'artists_ids': ['0yT9vbP1EOnCpl2McnF9nH'], 'artists_names': ['Mike Bloomfield'], 'album_id': '6bdy2PnssuzDkldvAOjVmj', 'album_name': 'Super Session (with Al Kooper & Stephen Stills)', 'album_release_date': '1968'}]


In [ ]:
man_alive_all_tracks_info = []
super_session_all_tracks_info = []

for album in man_alive_info:
    man_alive_all_tracks_info = get_all_tracks_from_album(album['album_id'], album) # Call function for each album in all_albums_info

for album in super_session_info:
    super_session_all_tracks_info = get_all_tracks_from_album(album['album_id'], album) # Call function for each album in all_albums_info

print(man_alive_all_tracks_info)

[{'artist_id': '4WlSvDKaq1PA2Nr7cCIPxX', 'artist_name': 'Stephen Stills', 'duration_ms': 203173, 'track_id': '1aayx0QsSgrcC4ZwKnTiko', 'track_name': "Ain't It Always", 'popularity': 0, 'album_id': '6nIuJOOeQf1OGpp7zzZWVx', 'album_name': 'Man Alive!', 'album_release_date': '2005-08-09', 'album_type': 'album'}, {'artist_id': '4WlSvDKaq1PA2Nr7cCIPxX', 'artist_name': 'Stephen Stills', 'duration_ms': 260933, 'track_id': '7F6LXINayj9PxiT7CSPAUI', 'track_name': 'Feed The People', 'popularity': 0, 'album_id': '6nIuJOOeQf1OGpp7zzZWVx', 'album_name': 'Man Alive!', 'album_release_date': '2005-08-09', 'album_type': 'album'}, {'artist_id': '2E6Roj0oQnJIm2BeXwDica', 'artist_name': 'Graham Nash', 'duration_ms': 260933, 'track_id': '7F6LXINayj9PxiT7CSPAUI', 'track_name': 'Feed The People', 'popularity': 0, 'album_id': '6nIuJOOeQf1OGpp7zzZWVx', 'album_name': 'Man Alive!', 'album_release_date': '2005-08-09', 'album_type': 'album'}, {'artist_id': '4WlSvDKaq1PA2Nr7cCIPxX', 'artist_name': 'Stephen Stills',

In [ ]:
audio_features_man_alive = get_audio_features_from_track(man_alive_all_tracks_info)
audio_features_super_session = get_audio_features_from_track(super_session_all_tracks_info)

### Create DataFrames

In [ ]:
# Create DataFrames from lists of dictionaries - Man Alive
all_tracks_info_df_man_alive = pd.DataFrame(man_alive_all_tracks_info)
audio_features_df_man_alive = pd.DataFrame(audio_features_man_alive)

# Merge the two DataFrames on 'track_id'
merged_df_man_alive = pd.merge(all_tracks_info_df_man_alive, audio_features_df_man_alive, on='track_id')
merged_df_man_alive

# Create DataFrames from lists of dictionaries - Super Session
all_tracks_info_df_super_session = pd.DataFrame(super_session_all_tracks_info)
audio_features_df_super_session = pd.DataFrame(audio_features_super_session)

# Merge the two DataFrames on 'track_id'
merged_df_super_session = pd.merge(all_tracks_info_df_super_session, audio_features_df_super_session, on='track_id')
merged_df_super_session

### Save DataFrame to a .csv file

In [ ]:
# Create CSV file. Check if file already exists
path_file = 'super-session.csv'
if os.path.isfile(path_file):
    print(f"File {path_file} already exists")
else:
    merged_df_super_session.to_csv(path_file)
    print(f"File {path_file} is successfully created")

File super-session.csv is successfully created


## The Stills-Young Band

In [ ]:
# Extract audio features for tracks: "Long May You Run", "Fontainebleau", "Let It Shine".
# Data for the rest of the songs isn't available on Spotify under the name "The Stills-Young Band"
long_may_you_run = '3DH8UjDQqj4QoN4HPWTkJq'
fontainebleau = '3PgCN6zhh9Cyy2i18Rf1dw'
let_it_shine = '5KSwKKIJpUDMukplC7lhjW'
# Some songs have only one artist listed:
make_love_to_you = '21erCP7YmcUMLZSorX5Nsk' # Stephen Stills Live version
midnight_on_the_bay = '4sKO5fXzCX3z3sIVBDcNLC' # Neil Young
ocean_girl = '7cmeO36ykPrL9HWFBk1KV2' # Neil Young

stills_young_band = [long_may_you_run, fontainebleau, let_it_shine, make_love_to_you, midnight_on_the_bay, ocean_girl]
stills_young_band_names = ["Long May You Run", "Fontainebleau", "Let It Shine", "Make Love to You", "Midnight on the Bay", "Ocean Girl"]

# Get audio features
audio_features_stills_young_band = sp.audio_features(stills_young_band)
print(audio_features_stills_young_band)

# Get track popularity
popularity_score = []
for song in stills_young_band:
  popularity = get_track_popularity(song)
  popularity_score.append(popularity)

stills_young_band_list = []
for i in range(len(stills_young_band)):
  features = audio_features_stills_young_band[i]
  stills_young_band_list.append({
        'track' : stills_young_band_names[i],
        'artist_name' : "The Stills-Young Band",
        'album_name' : "Long May You Run",
        'album_type' : "album",
        'album_release_date' : "1976-20-09",
        'danceability': features['danceability'],
        'energy': features['energy'],
        'key': features['key'],
        'loudness': features['loudness'],
        'mode': features['mode'],
        'speechiness': features['speechiness'],
        'acousticness': features['acousticness'],
        'instrumentalness': features['instrumentalness'],
        'liveness': features['liveness'],
        'valence': features['valence'],
        'tempo': features['tempo'],
        'popularity': popularity_score[i],
        'duration_ms' : features['duration_ms'],
        'time_signature' : features['time_signature']
  })

stills_young_band_df = pd.DataFrame(stills_young_band_list)
stills_young_band_df.to_csv('stills-young-band.csv')
stills_young_band_df

## Merge individual DataFrames to spotify.csv


In [ ]:
spotify_david_crosby_df = pd.read_csv('/content/drive/MyDrive/Eksplorativna analiza podataka sa Spotify-a/Spotify Web Scraping/Spotify Datasets/raw/spotify-david-crosby.csv')
spotify_stephen_stills_df = pd.read_csv('/content/drive/MyDrive/Eksplorativna analiza podataka sa Spotify-a/Spotify Web Scraping/Spotify Datasets/raw/spotify-stephen-stills.csv')
spotify_graham_nash_df = pd.read_csv('/content/drive/MyDrive/Eksplorativna analiza podataka sa Spotify-a/Spotify Web Scraping/Spotify Datasets/raw/spotify-graham-nash.csv')
spotify_neil_young_df = pd.read_csv('/content/drive/MyDrive/Eksplorativna analiza podataka sa Spotify-a/Spotify Web Scraping/Spotify Datasets/raw/spotify-neil-young.csv')
spotify_stills_young_band_df = pd.read_csv('/content/drive/MyDrive/Eksplorativna analiza podataka sa Spotify-a/Spotify Web Scraping/Spotify Datasets/raw/spotify-stills-young-band.csv')
spotify_cn_df = pd.read_csv('/content/drive/MyDrive/Eksplorativna analiza podataka sa Spotify-a/Spotify Web Scraping/Spotify Datasets/raw/spotify-crosby-nash.csv')
spotify_csn_df = pd.read_csv('/content/drive/MyDrive/Eksplorativna analiza podataka sa Spotify-a/Spotify Web Scraping/Spotify Datasets/raw/spotify-csn.csv')
spotify_csny_df = pd.read_csv('/content/drive/MyDrive/Eksplorativna analiza podataka sa Spotify-a/Spotify Web Scraping/Spotify Datasets/raw/spotify-csny.csv')

spotify_df = pd.concat([spotify_david_crosby_df,
                        spotify_stephen_stills_df,
                        spotify_graham_nash_df,
                        spotify_neil_young_df,
                        spotify_stills_young_band_df,
                        spotify_cn_df,
                        spotify_csn_df,
                        spotify_csny_df]).reset_index(drop=True)
spotify_df.to_csv('spotify.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
